`dask.array` Histogramming
==========================

`dask.array` provides histogramming via it's NumPy interface. ([N-dimensional histogram support was added quite recently!](https://github.com/dask/dask/issues/7307))

In [ ]:
import dask.array as da
x0 = da.random.standard_normal(size=(10_000_000,), chunks=(2_500_000,))
x0

In [ ]:
counts, edges = da.histogram(x0, bins=18, range=(-5, 5))
counts

In [ ]:
counts.visualize()

In [ ]:
result = counts.compute()
result

boost-histogram
===============

The [boost-histogram](https://boost-histogram.readthedocs.io/en/latest/)
library provides an object oriented API for histogramming in
Python; with analysis tooling via [Hist](https://hist.readthedocs.io/en/latest/) built around it as well. boost-histogram has more features and is more performant than NumPy-histogramming.

In [ ]:
import boost_histogram as bh
import numpy as np

In [ ]:
rng = np.random.default_rng(123)
x1 = rng.normal(3, 0.2, size=(5_000,))
y1 = rng.normal(5, 2.0, size=(5_000,))
w1 = rng.normal(4, 1.0, size=(5_000,))

In [ ]:
h1 = bh.Histogram(
    bh.axis.Regular(12, 2.5, 3.5),
    bh.axis.Regular(18, 0.0, 10.0),
    storage=bh.storage.Weight(),
)
h1 = h1.fill(x1, y1, weight=w1)
h1

In [ ]:
h2 = bh.numpy.histogram2d(x1, y1, bins=(12, 18), range=((2.5, 3.5), (0, 10)), weights=w1,
                          storage=bh.storage.Weight(), histogram=bh.Histogram)

In [ ]:
np.allclose(h1.counts(), h2.counts())

In [ ]:
h2

In [ ]:
import hist
h1 = hist.Hist(h1)
h1

In [ ]:
_ = h1.plot2d_full()

In [ ]:
_ = h1.project(0).plot1d()

dask-histogram
==============

Provides a Histogram object lazy-fillable with Dask collections.

In [ ]:
import dask_histogram as dh
x2 = da.random.normal(3, 0.2, size=(5_000_000,), chunks=(1_000_000,))
y2 = da.random.normal(5, 2.0, size=(5_000_000,), chunks=(1_000_000,))
w2 = da.random.normal(4, 1.0, size=(5_000_000,), chunks=(1_000_000,))

In [ ]:
h = dh.Histogram(
    dh.axis.Regular(12, 2.5, 3.5),
    dh.axis.Regular(18, 0.0, 10.0),
    storage=dh.storage.Weight(),
)
h.fill(x2, y2, weight=w2)

In [ ]:
h.empty()

In [ ]:
h.visualize()

In [ ]:
h.compute()

In [ ]:
hist.Hist(h)

In [ ]:
from typing import Tuple
from dask.delayed import delayed
import dask
import pickle

def data_factory(N: int = 10_000_000, chunks: int = 2_500_000) -> Tuple[da.Array]:
    x = da.random.standard_normal(size=(N,), chunks=(chunks,))
    w1 = da.random.uniform(0.80, 0.90, size=(N,), chunks=(chunks,))
    w2 = da.random.uniform(0.50, 0.70, size=(N,), chunks=(chunks,))
    w3 = da.random.uniform(0.10, 0.20, size=(N,), chunks=(chunks,))
    return x, w1, w2, w3

def save_histogram(hist: bh.Histogram, name: str) -> None:
    with open(f"{name}.pkl", "wb") as f:
        pickle.dump(hist, f)
        
delayed_save = delayed(save_histogram)

In [ ]:
x, w1, w2, w3 = data_factory()

h1 = dh.histogram(x, bins=30, range=(-3, 3), weights=w1, histogram=dh.Histogram, storage=dh.storage.Weight())
h2 = dh.histogram(x, bins=30, range=(-3, 3), weights=w2, histogram=dh.Histogram, storage=dh.storage.Weight())
h3 = dh.histogram(x, bins=30, range=(-3, 3), weights=w3, histogram=dh.Histogram, storage=dh.storage.Weight())

saves = [
    delayed_save(ih.to_delayed(), f"h{i}")
    for i, ih in enumerate([h1, h2, h3])
]

dask.visualize(saves)

In [ ]:
dask.compute(saves)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
for i in [0, 1, 2]:
    with open(f"h{i}.pkl", "rb") as f:
        h = pickle.load(f)
        hist.Hist(h).plot1d(ax=ax, label=f"hist{i}")
ax.legend()
plt.show()